In [2]:
import deeplake

from PIL import Image
from tqdm import tqdm
import numpy as np
import os
import time
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

/home/bdai/anaconda3/envs/deep_lake/lib/python3.9/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.17) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [3]:
def timing(start):
    print(f'Elapsed time: {time.time() - start:.2f} s')
# start = time.time()

In [12]:
image_path = "/home/bdai/spark_work/covid_dataset"
save_path = "/home/bdai/deeplake_work"
#train_data = deeplake.empty(save_path + "/deeplake_train")
test_data = deeplake.empty(save_path + "/deeplake_test")
# train_data = deeplake.load(save_path + "/deeplake_train")
# test_data = deeplake.load(save_path + "/deeplake_test")

/home/bdai/deeplake_work/deeplake_test loaded successfully.


In [5]:
def collect_files(path, data="train"):
    images = []
    for dir_path, dir_names, file_names in os.walk(path):
        for file_name in file_names:
            if os.path.splitext(file_name)[-1] == ".txt" and os.path.splitext(file_name)[0] == data:
                text = file_name
            else:
                path_end = os.path.split(dir_path)[-1]
                if path_end == data:
                    images.append(file_name)
    return images, text            

In [6]:
#train_images, train_text = collect_files(image_path, "train")
test_images, test_text = collect_files(image_path, "test")

class_names = ["positive", "negative"]

In [13]:
# label x 실험

# ds = train_data
# path = image_path+"/train"
# image_files = train_images
def image_transformation(ds, path, image_files, text_file, class_names):
# ds = train_data
# path = image_path+"/train"
# image_files = train_images
# text_file = train_text
    with ds:
        # Create the tensors with names of your choice.
        ds.create_tensor('images', htype = 'image', sample_compression = 'png')
        ds.create_tensor('file_id', sample_compression = "lz4")
        ds.create_tensor('labels', htype = 'class_label', class_names = class_names, sample_compression = "lz4")
        
        # Add arbitrary metadata - Optional
        ds.info.update(description = 'Covid classification dataset')
        info_data = pd.read_csv(image_path + "/" + text_file, sep = " ", names=["id", "file_name", "label", "orgin"], header=0)
        for image in tqdm(image_files):
            label_text = info_data.loc[info_data["file_name"] == image, "label"]
            if len(label_text) == 0 :
                continue
            label_num = class_names.index(label_text.values[0])
            ds.append({'images' : deeplake.read(os.path.join(path, image)), 'file_id' : image, 'labels': np.uint32(label_num)})
            #ds.append({'images' : deeplake.read(os.path.join(path, image)), 'file_id' : image})

In [14]:
start = time.time()
#image_transformation(train_data, image_path+"/train", train_images, train_text, class_names)
image_transformation(test_data, image_path+"/test", test_images, test_text, class_names)
timing(start)

100%|█████████████████████████████████████████| 400/400 [00:13<00:00, 28.69it/s]


Elapsed time: 14.08 s


In [109]:
spark_file = "/home/bdai/spark_work/spark-warehouse"
deeplake_file = "/home/bdai/deeplake_work"

def get_dir_size(path='.'):
    total = 0
    with os.scandir(path) as it:
        for entry in it:
            if entry.is_file():
                total += entry.stat().st_size
            elif entry.is_dir():
                total += get_dir_size(entry.path)
    return total
spark_dir_size = round(get_dir_size(spark_file) / (1024 ** 3),2)
deeplake_dir_size = round(get_dir_size(deeplake_file) / (1024 ** 3),2)

print("Total Paruet dataset size : {} GBs".format(spark_dir_size))
print("Total Tensor storage dataset size : {} GBs".format(deeplake_dir_size))

Total Paruet dataset size : 13.17 GBs
Total Tensor storage dataset size : 13.0 GBs
